In [ ]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import WikipediaAPIWrapper
from langchain.tools import WikipediaQueryRun
#duckduckgo가 계속 문제가 있어 대신 사용
from langchain.utilities import GoogleSearchAPIWrapper
import os


llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-0125")

class WikipediaResearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query for research based on websites searched on Wikipedia. Example: Research about springboot"
    )


class WikipediaResearchTool(BaseTool):
    name = "WikipediaResearchTool"
    description =  """
    Use this tool to research what you search for.
    It takes a query as an argument.
    """
    args_schema: Type[WikipediaResearchToolArgsSchema] = WikipediaResearchToolArgsSchema

    def _run(self, query):
        wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wiki.run(query)

class GoogleResearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query for research based on websites searched on Google. Example: Research about springboot"
    )


class GoogleResearchTool(BaseTool):
    name = "GoogleResearchTool"
    description = """
    Use this tool to research what you search for.
    It takes a query as an argument.
    """
    args_schema: Type[GoogleResearchToolArgsSchema] = GoogleResearchToolArgsSchema

    def _run(self, query):
        google = GoogleSearchAPIWrapper(
            google_api_key=os.environ.get("GOOGLE_API_KEY"),
            google_cse_id=os.environ.get("GOOGLE_CSE_ID")
        )
        return google.run(query)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaResearchTool(),
        GoogleResearchTool()
    ],
)

prompt = "Research about the XZ backdoor"
docs = agent.invoke(prompt)
print(docs)
with open("./research_results.txt", "w") as file:
    file.write(docs["output"])



> Entering new AgentExecutor chain...

Invoking: `WikipediaResearchTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution through OpenSSH on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.

Page: XZ Utils
Summary: XZ Uti